In [21]:
import matplotlib.pyplot as plt
import csv
import pandas as pd
import operator
from statistics import mean,pstdev
import numpy as np
import random
import copy

In [22]:
path="ml-latest-small/"
ratings_path=path+"ratings.csv"
movies_path=path+"movies.csv"

In [23]:
ratings_df=pd.read_csv(ratings_path)
ratings_df.head()


,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [24]:
movie_indices=[]
inverse_movie_map={}

In [25]:
movies_df=pd.read_csv(movies_path)
movies_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [26]:
for i in range(len(movies_df)):
    movie_indices.append(movies_df['movieId'][i])
    inverse_movie_map[movies_df['movieId'][i]]=i

In [30]:
number_of_items=len(movie_indices)
number_of_users={}
for i in range(len(ratings_df)):
    number_of_users[ratings_df['userId'][i]]=1

In [31]:
number_of_users=len(number_of_users)
user_sim_matrix_cosine=np.load(path+'user_sim_matrix_cosine.npy')
user_sim_matrix_pearson=np.load(path+'user_sim_matrix_pearson.npy')
ratings_matrix_train=np.load(path+'ratings_matrix_train.npy')
train_nos=np.load(path+'train_nos.npy')
test_nos=np.load(path+'test_nos.npy')
ratings_matrix_train_mean_centred=np.load(path+'ratings_matrix_train_mean_centred.npy')

In [32]:
correct_predictions=[]
for i in test_nos:
    correct_predictions.append(ratings_df['rating'][i])

In [33]:
def cosine_sim(a,b):
    mean_a=mean_with_nan(a)
    mean_b=mean_with_nan(b)
    #print(mean_a)
    #print(mean_b)
    num=0.0
    den_left=0.0
    den_right=0.0
    for i in range(len(a)):
        if (not np.isnan(a[i])) and (not np.isnan(b[i])):
            num+=(a[i])*(b[i])
            den_left+=(a[i])*(a[i])
            den_right+=(b[i])*(b[i])
    den_left=np.sqrt(den_left)
    #print(den_left)
    den_right=np.sqrt(den_right)
    #print(den_right)
    if (den_right==0 or den_left==0):
        return -1
    return num/((den_left)*(den_right))

In [34]:
def mean_absolute_error(a,b):
    error_sum=0.0
    for i in range(len(a)):
        error_sum+=abs(a[i]-b[i])
    error_sum/=len(a)
    return error_sum        

In [35]:
def rmse_error(a,b):
    error_sum=0.0
    for i in range(len(a)):
        error_sum+=(a[i]-b[i])*(a[i]-b[i])
    error_sum/=len(a)
    return np.sqrt(error_sum)     

In [61]:
k=2
pearson_raw_result=[]
for i in test_nos:
    movie_no=inverse_movie_map[ratings_df['movieId'][i]]
    user_no=ratings_df['userId'][i]-1
    
    top_k_similar_users_same_item_rated=[]
    for j in range(0,number_of_users):
        if(j!=user_no):
            if(not np.isnan(ratings_matrix_train[j][movie_no])):
                top_k_similar_users_same_item_rated.append((user_sim_matrix_pearson[user_no][j],ratings_matrix_train[j][movie_no]))
    #print(len(top_k_similar_users_same_item_rated))
    #print(top_k_similar_users_same_item_rated)            
    top_k_similar_users_same_item_rated=sorted(top_k_similar_users_same_item_rated,key=lambda x: x[0],reverse=True)  
    num=0.0
    den=0.0
    if(not len(top_k_similar_users_same_item_rated)<k):
        for p in range(0,k):
            num+=top_k_similar_users_same_item_rated[p][0]*top_k_similar_users_same_item_rated[p][1]
            den+=abs(top_k_similar_users_same_item_rated[p][0])
        pearson_raw_result.append(num/den) 
    else:
        print(ratings_df['movieId'][i])
        print(movie_no)
        print(user_no)
        print(i)
        print(top_k_similar_users_same_item_rated)
        print('\n')

1473
1126
0
96
[]


114060
8509
1
257
[]


7899
5052
2
295
[]


937
718
3
359
[(0.0734361327065403, 4.0)]


4252
3161
3
503
[]


189
160
5
637
[]


212
180
5
648
[(0.0004472364413267637, 3.5)]


219
187
5
652
[]


243
209
5
663
[]


510
445
5
784
[]


979
747
5
859
[]


5841
4088
8
1105
[(0.3981575634796399, 2.0)]


6044
4192
8
1118
[]


109853
8388
9
1250
[]


113275
8485
9
1252
[]


511
446
10
1284
[(0.5419833055558694, 2.0)]


8117
5102
17
2032
[]


64197
6913
17
2123
[(0.13420358295109583, 5.0)]


71108
7121
17
2141
[]


74948
7285
17
2155
[(0.2734351589991858, 4.0)]


85354
7555
17
2172
[(0.49073106023550633, 2.0)]


88094
7642
17
2179
[]


92094
7797
17
2189
[]


99728
8077
17
2201
[]


112421
8459
17
2223
[(0.16730904010470168, 4.5)]


114060
8509
17
2226
[]


145283
9114
17
2251
[]


155168
9244
17
2254
[]


157108
9267
17
2255
[]


158972
9302
17
2258
[(0.273887769615574, 4.0)]


422
366
18
2357
[(0.18033802160198428, 3.0)]


1324
1013
18
2508
[(0.2846737419683704, 0.5)]


148

[]


1144
866
89
14109
[]


384
341
90
14194
[(0.26038055991490744, 3.0)]


1771
1317
90
14371
[]


2298
1710
90
14420
[]


2651
1992
90
14447
[]


2782
2090
90
14464
[(0.20205186030528338, 4.5)]


2817
2120
90
14470
[(0.12925967106040462, 3.0)]


2992
2255
90
14484
[]


3840
2872
90
14542
[(0.3117444231855443, 1.0)]


4092
3056
90
14559
[]


4316
3200
90
14568
[(0.3413768320002695, 2.5)]


4599
3381
90
14578
[]


4626
3401
90
14580
[]


4909
3581
90
14595
[]


5165
3728
90
14602
[]


5346
3816
90
14605
[(0.2695387684714324, 1.0)]


6638
4489
90
14646
[(0.36541198754673654, 2.5)]


7492
4957
90
14672
[]


1659
1248
94
14924
[]


3840
2872
94
14970
[(1.0, 1.0)]


2337
1742
100
15458
[(0.30732213273070313, 3.0)]


57522
6669
102
15842
[]


62553
6869
102
15855
[]


64114
6910
102
15857
[]


77206
7316
102
15877
[]


77427
7322
102
15878
[]


81819
7463
102
15891
[(-0.2490243364235738, 4.0)]


91126
7750
102
15914
[]


91671
7779
102
15919
[(-0.041661559632691775, 1.0)]


3719
2780
103
16

70159
7086
176
25661
[]


71970
7169
176
25671
[]


77846
7332
176
25691
[]


981
748
180
26088
[(0.35270133672236154, 3.0)]


760
605
181
26210
[]


1437
1104
181
26331
[(0.1690027275302691, 5.0)]


1546
1166
181
26344
[]


1695
1278
181
26376
[(0.1690027275302691, 4.0)]


2310
1718
181
26459
[(0.27970383075849536, 4.0)]


2607
1964
181
26506
[(0.31231399054172826, 3.5)]


2696
2024
181
26517
[(0.26310675833869307, 5.0)]


2765
2081
181
26530
[]


3078
2324
181
26580
[(0.31231399054172826, 4.0)]


3150
2374
181
26590
[]


3158
2380
181
26591
[(0.1690027275302691, 3.0)]


3474
2598
181
26633
[]


3747
2805
181
26672
[(0.31231399054172826, 3.0)]


3761
2814
181
26676
[]


3895
2905
181
26697
[]


4152
3092
181
26752
[(0.5181977930686471, 4.5)]


4300
3191
181
26774
[(-0.09056123482166557, 4.0)]


4307
3195
181
26777
[]


4350
3221
181
26784
[(0.24533773653023905, 4.5)]


4699
3445
181
26811
[]


4826
3531
181
26827
[(0.3164121058553834, 4.0)]


4911
3582
181
26842
[]


5083
3690
181
268

48883
6336
253
37678
[]


117887
8603
255
37884
[(0.3919258262406701, 4.0)]


140725
9025
255
37898
[(0.19761159583868582, 3.0)]


128
110
259
37979
[]


4711
3454
259
38071
[(-0.005872991922183234, 4.0)]


7815
5025
259
38113
[]


8011
5084
259
38114
[(-0.24604101370029546, 4.0)]


8015
5087
259
38115
[]


988
753
262
38278
[(0.08398005485293511, 3.0)]


1415
1090
262
38300
[(0.08398005485293511, 4.0)]


4959
3611
262
38386
[(0.014342568448903876, 3.5)]


388
345
264
38521
[]


3194
2406
264
38638
[(0.30985451600161285, 3.5)]


2812
2116
265
38826
[]


1497
1143
273
39486
[(0.02309766095671255, 5.0)]


2514
1891
273
39652
[(0.29837821549959087, 0.5)]


2782
2090
273
39695
[(0.2779215146699294, 4.5)]


3990
2978
273
39850
[(-0.03391687062711704, 4.0)]


4224
3139
273
39878
[(0.04783314304329405, 3.0)]


4480
3311
273
39909
[(0.4298494885513678, 3.0)]


5165
3728
273
39966
[]


5569
3952
273
39993
[(0.25342094410014127, 3.0)]


5853
4092
273
40013
[]


6950
4651
273
40099
[(0.1101046662

42176
6096
317
49417
[]


42191
6097
317
49418
[]


43869
6134
317
49419
[]


47446
6269
317
49437
[]


48638
6322
317
49444
[]


60408
6788
317
49509
[(-0.1865017636274213, 5.0)]


60885
6808
317
49513
[(0.28628832052303643, 3.5)]


61394
6840
317
49519
[]


64167
6912
317
49530
[(-0.39111716938873115, 3.0)]


66511
6977
317
49542
[(-0.02662649375739541, 5.0)]


66744
6980
317
49543
[]


69953
7083
317
49568
[]


70984
7115
317
49571
[(0.09390213211238879, 2.0)]


71453
7143
317
49579
[]


74226
7249
317
49602
[]


79073
7370
317
49625
[]


81910
7468
317
49646
[(-0.0686501161541538, 4.0)]


85736
7566
317
49663
[(0.14965991065576043, 4.5)]


86237
7581
317
49664
[]


86290
7584
317
49666
[]


87444
7628
317
49675
[]


89030
7671
317
49681
[(0.2139962598542417, 3.5)]


90245
7709
317
49687
[]


90630
7732
317
49691
[]


90769
7738
317
49694
[]


91860
7784
317
49703
[]


97168
7998
317
49725
[]


97904
8021
317
49731
[]


101741
8135
317
49741
[(0.17912916845753069, 3.5)]


105197
826

7208
4829
389
60377
[]


4273
3173
390
60742
[(0.4646979496973249, 4.0)]


5477
3899
390
60787
[(0.4069404882335952, 3.0)]


5575
3957
390
60790
[]


6408
4375
397
61101
[]


115664
8547
400
61262
[(0.16614547798964296, 4.5)]


61697
6846
402
61351
[]


70708
7110
404
61529
[]


6776
4561
407
61623
[(0.23639605809364725, 4.5)]


87785
7635
407
61678
[]


110669
8412
407
61698
[(-0.3703208088734787, 4.0)]


3590
2680
408
61829
[]


670
557
409
61865
[(0.182808469497841, 4.5)]


824
641
409
61868
[]


3823
2857
409
61976
[]


4256
3164
409
61999
[(0.028994928668389562, 3.0)]


408
353
413
62470
[]


635
538
413
62568
[(0.2534596039797947, 3.0)]


1040
795
413
62677
[]


1121
851
413
62705
[]


1484
1134
413
62834
[(0.5372224405747635, 3.0)]


1870
1368
413
62927
[]


2044
1515
413
62983
[]


2453
1844
413
63140
[(0.31005005803258545, 1.5)]


2472
1860
413
63149
[(0.4413485044936523, 3.0)]


2552
1924
413
63169
[(0.5372224405747635, 1.0)]


2766
2082
413
63226
[(0.1915632997303834, 3.5)]


70223
[]


110826
8419
447
70227
[]


111663
8434
447
70234
[]


112303
8454
447
70244
[]


113186
8481
447
70256
[]


113225
8483
447
70257
[]


113348
8490
447
70258
[(0.5269014214313085, 2.5)]


113350
8491
447
70259
[]


114060
8509
447
70265
[]


114762
8528
447
70270
[(0.18392614756370193, 3.5)]


115170
8539
447
70276
[]


115216
8542
447
70278
[]


115231
8543
447
70279
[(0.250751321612865, 2.5)]


116207
8559
447
70283
[]


116413
8561
447
70284
[]


116419
8562
447
70285
[]


116849
8573
447
70287
[]


117107
8580
447
70291
[]


118814
8622
447
70304
[]


118985
8632
447
70307
[]


119655
8641
447
70313
[]


120637
8651
447
70316
[(0.4017831397824433, 3.0)]


122932
8700
447
70329
[]


124859
8711
447
70331
[]


127194
8743
447
70338
[]


127204
8746
447
70340
[]


128592
8763
447
70343
[]


129937
8793
447
70351
[(0.5269014214313085, 3.0)]


131480
8823
447
70356
[]


133195
8855
447
70365
[]


133365
8858
447
70367
[]


133377
8859
447
70368
[]


134158
8880
447
70375
[]




48598
6321
473
75154
[]


49957
6372
473
75169
[(0.3855100698076644, 3.0)]


50160
6380
473
75172
[(-0.005088085620446174, 4.0)]


50514
6388
473
75174
[]


53921
6515
473
75185
[]


55071
6565
473
75190
[]


5069
3680
476
75790
[(0.42635516836880205, 2.5)]


6584
4460
476
75845
[(0.28999980448459184, 2.5)]


53574
6510
476
75955
[(0.23467278577064546, 4.0)]


72224
7179
476
76011
[(0.23377777875809436, 1.0)]


82242
7480
476
76023
[(0.3640592482771864, 3.5)]


4092
3056
478
76180
[]


4111
3064
478
76181
[]


4471
3304
478
76190
[(0.7017488279274923, 1.0)]


4488
3316
478
76191
[]


4831
3535
478
76202
[]


5448
3876
478
76217
[]


5651
4004
478
76224
[]


4727
3466
479
76691
[(0.4140700449206053, 3.5)]


5483
3904
479
76754
[(0.33623791205142983, 4.5)]


6853
4602
479
76817
[]


7618
4976
479
76855
[(0.33623791205142983, 3.5)]


8131
5109
479
76857
[]


32294
5831
479
76924
[]


43914
6139
479
77007
[]


44197
6154
479
77011
[]


47646
6280
479
77043
[]


49276
6348
479
77052
[(0.382

1324
1013
570
88430
[(0.3122718126221557, 0.5)]


1623
1222
570
88449
[(0.41386059423091764, 2.5)]


1978
1455
570
88459
[]


2465
1854
570
88486
[(0.41386059423091764, 1.5)]


3073
2319
571
88667
[]


46347
6236
572
88895
[(0.3614572079313923, 3.5)]


2538
1913
574
89010
[]


2623
1976
574
89033
[]


6751
4545
577
89222
[]


52668
6466
577
89230
[(0.4278333295046984, 4.0)]


56389
6640
577
89231
[]


5767
4051
579
89601
[]


5828
4081
579
89605
[]


6300
4312
579
89619
[(0.5086584032345366, 4.5)]


7579
4969
579
89662
[(-0.035071642994025945, 3.5)]


8092
5099
579
89669
[]


8427
5188
579
89678
[]


8730
5286
579
89683
[(-0.035071642994025945, 3.5)]


8800
5307
579
89685
[]


26819
5582
579
89695
[]


44193
6152
582
89886
[(0.10298833091790689, 3.0)]


110130
8397
585
90209
[(0.18075784361208125, 4.0)]


136556
8946
585
90238
[]


766
610
587
90475
[(-1.0, 4.0)]


6357
4349
589
91078
[(0.4301317596327325, 3.0)]


7056
4734
589
91104
[(0.22907366034642113, 4.5)]


6776
4561
592
91479
[

3077
2323
599
95556
[(0.050020884108937576, 4.0)]


3964
2956
599
95617
[(0.17171462600205095, 3.0)]


6319
4324
599
95725
[(0.24885596036680213, 1.0)]


6750
4544
599
95738
[(0.40062598693746543, 2.0)]


25905
5428
599
95787
[]


25947
5435
599
95789
[]


26084
5451
599
95790
[(0.18804769079526729, 3.5)]


26326
5489
599
95793
[(0.6549752208990437, 4.0)]


27563
5659
599
95799
[(0.10319779772364215, 4.5)]


46664
6243
599
95836
[]


50740
6394
599
95847
[(0.19480711985495147, 4.0)]


172591
9541
600
95961
[(0.35521174144853473, 5.0)]


422
366
601
96054
[(0.22974460374552477, 3.0)]


108
96
602
96127
[]


149
122
602
96135
[]


156
129
602
96137
[(0.27397129613680876, 4.0)]


388
345
602
96200
[]


568
493
602
96240
[(0.20987948572805487, 4.0)]


766
610
602
96267
[(-0.06951889113890922, 4.0)]


1053
804
602
96322
[]


1112
845
602
96343
[(0.2903074121745651, 4.0)]


1176
881
602
96356
[]


1419
1093
602
96466
[]


1484
1134
602
96475
[(0.17545130419065755, 3.0)]


1497
1143
602
96476

In [63]:
def mean_with_nan(list_a):
    count=0
    sum=0.0
    for i in list_a:
        if not np.isnan(i):
            #print(i)
            count+=1
            sum+=i
    #print(sum) 
    if(count==0):
        return 0
    return sum/count        
    

In [59]:
inverse_movie_map[1408]

1084

In [ ]:
item_sim_matrix_cosine=np.zeros(shape=(number_of_items,number_of_items))
item_sim_matrix_cosine.fill(np.nan)
for i in range(0,number_of_items):
    for j in range(i,number_of_items):
        item_sim_matrix_cosine[i][j]=cosine_sim(ratings_matrix_train[:,i],ratings_matrix_train[:,j])
        item_sim_matrix_cosine[j][i]=item_sim_matrix_cosine[i][j]
np.save(path+'item_sim_matrix_cosine',item_sim_matrix_cosine)        